<h3><u>FORMING THE DATA</u></h3>

In [1]:
# Import the necessary packages
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn import preprocessing

In [2]:
# Getting train and test data
df=pd.read_excel('Data_Train.xlsx')
df_test=pd.read_excel('Data_Test.xlsx')

<h3><u>DATA CLEANING</u></h3>

In [3]:
# Removing cars with exceptional values 
df=df[df['Name'] != 'Ambassador Classic Nova Diesel']
df=df[df['Name'] != 'Lamborghini Gallardo Coupe']
df=df[df['Name'] != 'Force One LX 4x4']
df=df[df['Name'] != 'Force One LX ABS 7 Seating']
df=df[df['Name'] != 'Smart Fortwo CDI AT']
df = df[df['Fuel_Type'] != 'Electric']

In [4]:
# Splitting Name column into 2 different columns
df['model']=df['Name'].apply(lambda x : str(x).split(' ')[1:])
df_test['model']=df_test['Name'].apply(lambda x : str(x).split(' ')[1:])
df['model']=df['model'].apply(lambda x : ' '.join(map(str, x)))
df_test['model']=df_test['model'].apply(lambda x : ' '.join(map(str, x)))

df['brand']=df['Name'].apply(lambda x : str(x).split(' ')[0])
df_test['brand']=df_test['Name'].apply(lambda x : str(x).split(' ')[0])

In [5]:
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,model,brand
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75,Wagon R LXI CNG,Maruti
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50,Creta 1.6 CRDi SX Option,Hyundai
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50,Jazz V,Honda
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00,Ertiga VDI,Maruti
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74,A4 New 2.0 TDI Multitronic,Audi


In [6]:
# Defining car age 
df['Car_Age'] = 2020 - df['Year']
df.drop('Year', axis=1, inplace=True)

In [7]:
# creating temporary dataframe to get more data
temp=pd.concat([df,df_test],ignore_index=False,axis=0)

In [8]:
temp['Engine'].mode()

0    1197 CC
dtype: object

In [9]:
temp['Power'].mode()

0    74 bhp
dtype: object

In [10]:
temp['Mileage'].mode()

0    17.0 kmpl
dtype: object

In [11]:
# Replacing null values and cleaning the columns
df['Engine'] = df['Engine'].apply(lambda x : str(x).split(" ")[0]).astype(float)
df['Power'] = df['Power'].replace('null bhp',None).apply(lambda x : str(x).split(' ')[0]).astype(float)
df['Mileage'] = df['Mileage'].replace('0.0 kmpl',None).apply(lambda x : str(x).split(' ')[0]).astype(float)

df['Engine'] = df.transform(lambda x: x.fillna(1197))['Engine']
df['Power'] = df.transform(lambda x: x.fillna(74))['Power']
df['Mileage'] = df.transform(lambda x: x.fillna(17))['Mileage']

In [12]:
# Label encoding
label_encoder = preprocessing.LabelEncoder()

df['model'] = label_encoder.fit_transform(df['model'])

df['brand'] = label_encoder.fit_transform(df['brand'])

df['Location'] = label_encoder.fit_transform(df['Location'])

df['Fuel_Type'] = label_encoder.fit_transform(df['Fuel_Type'])

df['Transmission'] = label_encoder.fit_transform(df['Transmission'])

df['Owner_Type'] = label_encoder.fit_transform(df['Owner_Type'])

In [13]:
# Dropping outliers
df.drop(df[(df['Engine']>=5500) | (df['Engine']<=600)].index,inplace=True)
df.drop(df[(df['Kilometers_Driven']>700000) | (df['Kilometers_Driven']<=999)].index,inplace=True)
df.drop(df[(df['Seats']==0)].index,inplace=True)
df.drop('Name',axis=1,inplace=True)
df.dropna(subset=['Seats'],how='any',inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962 entries, 0 to 6018
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Location           5962 non-null   int32  
 1   Kilometers_Driven  5962 non-null   int64  
 2   Fuel_Type          5962 non-null   int32  
 3   Transmission       5962 non-null   int32  
 4   Owner_Type         5962 non-null   int32  
 5   Mileage            5962 non-null   float64
 6   Engine             5962 non-null   float64
 7   Power              5962 non-null   float64
 8   Seats              5962 non-null   float64
 9   Price              5962 non-null   float64
 10  model              5962 non-null   int32  
 11  brand              5962 non-null   int32  
 12  Car_Age            5962 non-null   int64  
dtypes: float64(5), int32(6), int64(2)
memory usage: 512.4 KB


In [15]:
df.to_csv('df_clean.csv', index=False)

<h3><u>XGBoost Application</u></h3>

In [16]:
# Defining Dependent and independent variables
x = df.drop(labels=['Price'], axis=1)
y = df['Price'].values

from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.25, random_state=1)

In [17]:
df.head()

,Location,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,model,brand,Car_Age
0,9,72000,0,1,0,26.60,998.0,58.16,5.0,1.75,1636,15,10
1,10,41000,1,1,0,19.67,1582.0,126.20,5.0,12.50,459,8,5
2,2,46000,3,1,0,18.20,1199.0,88.70,5.0,4.50,907,7,9
3,2,87000,1,1,0,20.77,1248.0,88.76,7.0,6.00,618,15,8
4,3,40670,1,0,2,15.20,1968.0,140.80,5.0,17.74,96,0,7


In [18]:
from xgboost import XGBRegressor

my_model = XGBRegressor(n_estimators=500, learning_rate=0.25, max_depth=5,subsample=1,min_child_weight=17)
my_model.fit(xTrain, yTrain, 
             early_stopping_rounds=5, 
             eval_set=[(xTest, yTest)], 
             verbose=False)

yPrediction = my_model.predict(xTest)

In [19]:
r2_score(yTest, yPrediction)

0.9307665964121966

<h3><u>Test Data Analysis</u></h3>

In [20]:
# Replacing null values and cleaning the columns
df_test['Engine'] = df_test['Engine'].apply(lambda x : str(x).split(" ")[0]).astype(float)
df_test['Power'] = df_test['Power'].replace('null bhp',None).apply(lambda x : str(x).split(' ')[0]).astype(float)
df_test['Mileage'] = df_test['Mileage'].replace('0.0 kmpl',None).apply(lambda x : str(x).split(' ')[0]).astype(float)

df_test['Engine'] = df_test.transform(lambda x: x.fillna(1197))['Engine']
df_test['Power'] = df_test.transform(lambda x: x.fillna(74))['Power']
df_test['Mileage'] = df_test.transform(lambda x: x.fillna(17))['Mileage']

In [21]:
# Defining car age 
df_test['Car_Age'] = 2020 - df_test['Year']
df_test.drop('Year', axis=1, inplace=True)

In [22]:
# Label encoding
label_encoder = preprocessing.LabelEncoder()

df_test['model'] = label_encoder.fit_transform(df_test['model'])

df_test['brand'] = label_encoder.fit_transform(df_test['brand'])

df_test['Location'] = label_encoder.fit_transform(df_test['Location'])

df_test['Fuel_Type'] = label_encoder.fit_transform(df_test['Fuel_Type'])

df_test['Transmission'] = label_encoder.fit_transform(df_test['Transmission'])

df_test['Owner_Type'] = label_encoder.fit_transform(df_test['Owner_Type'])

In [23]:
df_test.drop('Name',axis=1,inplace=True)

In [24]:
df_test.head()

,Location,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,model,brand,Car_Age
0,4,40929,0,1,0,32.26,998.0,58.20,4.0,70,16,6
1,3,54493,3,1,2,24.70,796.0,47.30,5.0,65,16,7
2,9,34000,1,1,0,13.68,2393.0,147.80,7.0,398,26,3
3,5,139000,1,1,0,23.59,1364.0,147.80,5.0,298,26,8
4,9,29000,3,1,0,18.50,1197.0,82.85,5.0,759,9,6


In [25]:
df_test.to_csv('df_test_clean.csv', index=False)

In [26]:
yPrediction = my_model.predict(df_test)
df_test=pd.read_excel('Data_Test.xlsx')
result=df_test.copy()
result=pd.concat([result,pd.Series(yPrediction,name='Price')],axis=1)
result

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Alto K10 LXI CNG,Delhi,2014,40929,CNG,Manual,First,32.26 km/kg,998 CC,58.2 bhp,4.0,6.442803
1,Maruti Alto 800 2016-2019 LXI,Coimbatore,2013,54493,Petrol,Manual,Second,24.7 kmpl,796 CC,47.3 bhp,5.0,3.447170
2,Toyota Innova Crysta Touring Sport 2.4 MT,Mumbai,2017,34000,Diesel,Manual,First,13.68 kmpl,2393 CC,147.8 bhp,7.0,17.420637
3,Toyota Etios Liva GD,Hyderabad,2012,139000,Diesel,Manual,First,23.59 kmpl,1364 CC,null bhp,5.0,7.906877
4,Hyundai i20 Magna,Mumbai,2014,29000,Petrol,Manual,First,18.5 kmpl,1197 CC,82.85 bhp,5.0,3.990642
...,...,...,...,...,...,...,...,...,...,...,...,...
1229,Volkswagen Vento Diesel Trendline,Hyderabad,2011,89411,Diesel,Manual,First,20.54 kmpl,1598 CC,103.6 bhp,5.0,4.063383
1230,Volkswagen Polo GT TSI,Mumbai,2015,59000,Petrol,Automatic,First,17.21 kmpl,1197 CC,103.6 bhp,5.0,6.889301
1231,Nissan Micra Diesel XV,Kolkata,2012,28000,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,4.897276
1232,Volkswagen Polo GT TSI,Pune,2013,52262,Petrol,Automatic,Third,17.2 kmpl,1197 CC,103.6 bhp,5.0,5.279183


In [27]:
result.to_csv('Predicted_Prices.csv', index=False)